In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from networks import NoKafnet, Kafnet
import utils.datasetsUtils.CIFAR as CIFAR
from utils.datasetsUtils.taskManager import SingleTargetClassificationTask, NoTask
import configs.configClasses as configClasses
from torchvision.transforms import transforms
import torch
from Trainer import Trainer
import matplotlib.pyplot as plt
from collections import defaultdict
import copy
import numpy as np

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

True
1
GeForce GTX 1050


In [4]:
config = configClasses.OnlineLearningConfig()
config.EPOCHS = 20
config.L1_REG = 0
config.IS_CONVOLUTIONAL = True
config.SAVE_PATH = './models/cifar10/kaf-vs-nokaf'
config.MODEL_NAME = 'ewc'
print(config)

confing_no_ewt = copy.copy(config)
confing_no_ewt.USE_EWC = True
confing_no_ewt.MODEL_NAME = 'no_ewc'
print(confing_no_ewt)

confing_no_ewt_kaf = copy.copy(config)
confing_no_ewt_kaf.MODEL_NAME = 'no_ewc_kaf'
confing_no_ewt_kaf.USE_EWC = True
print(confing_no_ewt_kaf)

confing_ewt_kaf = copy.copy(config)
confing_ewt_kaf.MODEL_NAME = 'ewc_kaf'
confing_ewt_kaf.EWC_IMPORTANCE = 500
print(confing_ewt_kaf)

CONFIG PARAMETERS
BATCH_SIZE: 64
DEVICE: cuda
EPOCHS: 20
EWC_IMPORTANCE: 1000
EWC_SAMPLE_SIZE: 250
EWC_TYPE: <class 'networks.continual_learning.OnlineEWC'>
GAMMA: 1.0
IS_CONVOLUTIONAL: True
ITERS: 1
L1_REG: 0
LOSS: cross_entropy
LR: 0.001
MODEL_NAME: ewc
OPTIMIZER: SGD
RUN_NAME: default
SAVE_PATH: ./models/cifar10/kaf-vs-nokaf
USE_EWC: True
USE_TENSORBOARD: True

CONFIG PARAMETERS
BATCH_SIZE: 64
DEVICE: cuda
EPOCHS: 20
EWC_IMPORTANCE: 1000
EWC_SAMPLE_SIZE: 250
EWC_TYPE: <class 'networks.continual_learning.OnlineEWC'>
GAMMA: 1.0
IS_CONVOLUTIONAL: True
ITERS: 1
L1_REG: 0
LOSS: cross_entropy
LR: 0.001
MODEL_NAME: no_ewc
OPTIMIZER: SGD
RUN_NAME: default
SAVE_PATH: ./models/cifar10/kaf-vs-nokaf
USE_EWC: True
USE_TENSORBOARD: True

CONFIG PARAMETERS
BATCH_SIZE: 64
DEVICE: cuda
EPOCHS: 20
EWC_IMPORTANCE: 1000
EWC_SAMPLE_SIZE: 250
EWC_TYPE: <class 'networks.continual_learning.OnlineEWC'>
GAMMA: 1.0
IS_CONVOLUTIONAL: True
ITERS: 1
L1_REG: 0
LOSS: cross_entropy
LR: 0.001
MODEL_NAME: no_ewc_kaf


In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
)

dataset = CIFAR.Cifar10('../data/cifar10', SingleTargetClassificationTask(), download=True,
                        force_download=False, train_split=0.8, transform=transform, target_transform=None)
dataset.load_dataset()

../data/cifar10/download
task #0 with train 48000 and test 12000 images (label: airplane)
task #1 with train 48000 and test 12000 images (label: automobile)
task #2 with train 48000 and test 12000 images (label: bird)
task #3 with train 48000 and test 12000 images (label: cat)
task #4 with train 48000 and test 12000 images (label: deer)
task #5 with train 48000 and test 12000 images (label: dog)
task #6 with train 48000 and test 12000 images (label: frog)
task #7 with train 48000 and test 12000 images (label: horse)
task #8 with train 48000 and test 12000 images (label: ship)
task #9 with train 48000 and test 12000 images (label: truck)


In [6]:
net_ewt = NoKafnet.CNN(dataset.tasks_number)
net_no_ewt = copy.deepcopy(net_ewt)

kaf_ewt = Kafnet.KAFCNN(dataset.tasks_number)
kaf_no_ewt = copy.deepcopy(kaf_ewt)#Kafnet.KAFMLP(len(dataset.class_to_idx))

print('Numero di parametri rete classica: ', sum([torch.numel(p) for p in net_ewt.parameters()]))
print('Numero di parametri KAFNET: ', sum([torch.numel(p) for p in kaf_ewt.parameters()]))

Numero di parametri rete classica:  171732
Numero di parametri KAFNET:  120020


In [7]:
trainer_kaf_ewt = Trainer(kaf_ewt, copy.deepcopy(dataset), confing_ewt_kaf)
trainer_kaf_no_ewt = Trainer(kaf_no_ewt, copy.deepcopy(dataset), confing_no_ewt_kaf)

trainer_ewt = Trainer(net_ewt, copy.deepcopy(dataset), config)
trainer_no_ewt = Trainer(net_no_ewt, copy.deepcopy(dataset), confing_no_ewt)

In [8]:
metrics_ewt = trainer_ewt.load()
if not metrics_ewt:
    metrics_ewt = trainer_ewt.all_tasks()

In [9]:
metrics_no_ewt = trainer_no_ewt.load()
if not metrics_no_ewt:
    metrics_no_ewt = trainer_no_ewt.all_tasks()

In [10]:
metrics_kaf_ewt = trainer_kaf_ewt.load()
if not metrics_kaf_ewt:
    metrics_kaf_ewt = trainer_kaf_ewt.all_tasks()

Training task (ewc) 0, epoch 1: 100%|██████████| 750/750 [02:22<00:00,  5.15it/s, loss=0.268, batch#=750]
Testing task 0: 188it [00:15, 12.62it/s, batch#=188]
Training task (ewc) 0, epoch 2: 100%|██████████| 750/750 [02:28<00:00,  5.43it/s, loss=0.227, batch#=750]
Testing task 0: 188it [00:15, 12.87it/s, batch#=188]
Training task (ewc) 0, epoch 3: 100%|██████████| 750/750 [02:37<00:00,  5.48it/s, loss=0.215, batch#=750]
Testing task 0: 188it [00:21,  8.90it/s, batch#=188]
Training task (ewc) 0, epoch 4: 100%|██████████| 750/750 [02:41<00:00,  6.21it/s, loss=0.207, batch#=750]
Testing task 0: 188it [00:13, 15.30it/s, batch#=188]
Training task (ewc) 0, epoch 5: 100%|██████████| 750/750 [01:55<00:00,  6.54it/s, loss=0.2, batch#=750]  
Testing task 0: 188it [00:12, 15.64it/s, batch#=188]
Training task (ewc) 0, epoch 6: 100%|██████████| 750/750 [01:57<00:00,  6.35it/s, loss=0.195, batch#=750]
Testing task 0: 188it [00:13, 14.80it/s, batch#=188]
Training task (ewc) 0, epoch 7: 100%|█████████

Testing task 7: 188it [00:13, 14.65it/s, batch#=188]
Testing task 8: 188it [00:13, 14.67it/s, batch#=188]
Testing task 9: 188it [00:13, 14.70it/s, batch#=188]
Training task (ewc) 2, epoch 1: 100%|██████████| 750/750 [02:00<00:00,  6.26it/s, loss=0.313, batch#=750]
Testing task 2: 188it [00:13, 14.65it/s, batch#=188]
Testing task 0: 188it [00:13, 14.66it/s, batch#=188]
Testing task 1: 188it [00:13, 14.68it/s, batch#=188]
Training task (ewc) 2, epoch 2: 100%|██████████| 750/750 [02:00<00:00,  6.23it/s, loss=0.381, batch#=750]
Testing task 2: 188it [00:13, 14.60it/s, batch#=188]
Testing task 0: 188it [00:13, 14.60it/s, batch#=188]
Testing task 1: 188it [00:13, 14.68it/s, batch#=188]
Training task (ewc) 2, epoch 3: 100%|██████████| 750/750 [02:00<00:00,  6.26it/s, loss=0.367, batch#=750]
Testing task 2: 188it [00:13, 14.68it/s, batch#=188]
Testing task 0: 188it [00:13, 14.66it/s, batch#=188]
Testing task 1: 188it [00:13, 14.60it/s, batch#=188]
Training task (ewc) 2, epoch 4: 100%|█████████

KeyboardInterrupt: 

In [ ]:
metrics_kaf_no_ewt = trainer_kaf_no_ewt.load()
if not metrics_kaf_no_ewt:
    metrics_kaf_no_ewt = trainer_kaf_no_ewt.all_tasks()

In [ ]:
n_task = len(metrics_ewt['tasks'])
tot_epochs = 0

print('No ewc', metrics_no_ewt['metrics'])
print('Online ewc', metrics_ewt['metrics'])

print('KAF no ewc', metrics_kaf_no_ewt['metrics'])
print('KAF online ewc', metrics_kaf_ewt['metrics'])

for k, v in metrics_no_ewt['tasks'].items():
    tot_epochs = max(tot_epochs, len(v['accuracy']))
             
for k, v in metrics_ewt['tasks'].items():
    tot_epochs = max(tot_epochs, len(v['accuracy']))
      
fig = plt.figure(figsize=(12, 24))

ax = None
for i, task in enumerate(metrics_ewt['tasks'].keys()):
        
    ewt = metrics_ewt['tasks'][task]
    no_ewt = metrics_no_ewt['tasks'][task]
    
    kaf_ewt = metrics_kaf_ewt['tasks'][task]
    kaf_no_ewt = metrics_kaf_no_ewt['tasks'][task]
    

    x = range(tot_epochs-len(ewt['accuracy']), tot_epochs)

    ax = fig.add_subplot(n_task, 1, i+1, sharex=ax) 
    
    ax.plot(x, ewt['f1'], label='ewt')
    ax.plot(x, no_ewt['f1'], label='no ewt')
    ax.plot(x, kaf_ewt['f1'], label='kaf ewt')
    ax.plot(x, kaf_no_ewt['f1'], label='kaf no ewt')
    
    ax.set_xticks(list(range(0, tot_epochs, 2))+[39],minor=False)
    
    ax.set_title("Task {}".format(task))
    ax.legend(loc="lower left")
    ax.grid(True, axis='x')
    
fig.subplots_adjust(hspace=0.01)
